In [1]:
import os
import tqdm
from PIL import Image
data = []

for file in os.listdir("dataset"):
	if "t" not in file:
		data.append(file)

inputs = []
outputs = []

for image in data:
	img_source	= Image.open(f"dataset/{image}")
	img_target 	= Image.open(f"dataset/target_{image}")

	width, height = img_source.size

	source_rgb = img_source.convert("RGB")
	target_rgb = img_target.convert("RGB")

	for x in tqdm.tqdm(range(width), desc=image):
		for y in range(height):
			r,g,b = target_rgb.getpixel((x,y))
			target = 0.01
			if r == g == b == 255:
				target = 0.99

			r,g,b = source_rgb.getpixel((x,y))

			inputs.append([r/255,g/255,b/255])
			outputs.append(target)

panneau5.jpg: 100%|██████████| 1920/1920 [00:06<00:00, 298.35it/s]


In [20]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

outputs = np.array(outputs, dtype=float)
inputs = np.array(inputs, dtype=float)


print(f"{np.count_nonzero(outputs == 0.99) / len(outputs) * 100} %")

model = keras.Sequential(
	[
		layers.Dense(3, input_shape=[3]),
		layers.Dense(3, activation="sigmoid", name="hidden"),
		layers.Dense(1, activation="sigmoid", name="ouput")
	]
)

model.compile(
	optimizer=keras.optimizers.Adam(0.025),
	loss='mean_squared_error'
)

model.summary()

7.1126580996913935 %
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 3)                 12        
                                                                 
 hidden (Dense)              (None, 3)                 12        
                                                                 
 ouput (Dense)               (None, 1)                 4         
                                                                 
Total params: 28
Trainable params: 28
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.fit(
	inputs,
	outputs,
	batch_size=50000,
	epochs=50
)

Epoch 1/50
193/193 [==============================] - 2s 6ms/step - loss: 0.0736
Epoch 2/50
193/193 [==============================] - 1s 6ms/step - loss: 0.0486
Epoch 3/50
193/193 [==============================] - 1s 6ms/step - loss: 0.0161
Epoch 4/50
193/193 [==============================] - 1s 6ms/step - loss: 0.0148
Epoch 5/50
193/193 [==============================] - 1s 6ms/step - loss: 0.0143
Epoch 6/50
193/193 [==============================] - 1s 6ms/step - loss: 0.0141
Epoch 7/50
193/193 [==============================] - 1s 6ms/step - loss: 0.0140
Epoch 8/50
193/193 [==============================] - 1s 6ms/step - loss: 0.0140
Epoch 9/50
193/193 [==============================] - 1s 6ms/step - loss: 0.0139
Epoch 10/50
193/193 [==============================] - 1s 6ms/step - loss: 0.0139
Epoch 11/50
193/193 [==============================] - 1s 6ms/step - loss: 0.0139
Epoch 12/50
193/193 [==============================] - 1s 6ms/step - loss: 0.0139
Epoch 13/50
193/193 [====

In [ ]:
model.save("new_model.h5")